In [1]:
import torch
from copy import deepcopy
from torch import nn
from torch.ao.nn.quantized.functional import threshold

In [2]:
torch.manual_seed(0);

In [3]:
from senmodel.model.model import *

In [4]:
def dense_to_sparse(dense_tensor: torch.Tensor) -> torch.Tensor:
    indices = dense_tensor.nonzero(as_tuple=True)
    values = dense_tensor[indices]
    indices = torch.stack(indices)

    sparse_tensor = torch.sparse_coo_tensor(indices, values, dense_tensor.size())
    return sparse_tensor

In [5]:
def convert_dense_to_sparse_network(model: nn.Module) -> nn.Module:
    """
    Converts a given dense neural network model to a sparse neural network model.

    This function recursively iterate through the given model and replaces all instances of 
    `nn.Linear` layers with `SparseLinear` layers

    Args:
        model (nn.Module): The dense neural network model to be converted.

    Returns:
        nn.Module: A new neural network model with sparse layers.
    """
    new_model = model.__class__()

    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            sparse_weight = dense_to_sparse(module.weight.data)
            sparse_bias = dense_to_sparse(module.bias.data)

            setattr(new_model, name, ExpandingLinear(sparse_weight, sparse_bias))
        else:
            setattr(new_model, name, convert_dense_to_sparse_network(module))
    return new_model

In [6]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=8):
        super(SimpleFCN, self).__init__()
        self.fc1 = nn.Linear(input_size, 4)
        self.fc2 = nn.Linear(4, 4)
        self.fc3 = nn.Linear(2, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = SimpleFCN()

In [7]:
sparse_model = convert_dense_to_sparse_network(model)

In [8]:
linear = nn.Linear(8, 1)

sparse_weight = dense_to_sparse(linear.weight.data)
sparse_bias = dense_to_sparse(linear.bias.data)
just_sparse_linear = ExpandingLinear(sparse_weight, sparse_bias)

sparse_linear = deepcopy(just_sparse_linear)

print(sparse_linear.weight_indices, "\n")
sparse_linear.replace(0, 6, 0)
sparse_linear.replace(0, 7, 0)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[0].weight_indices, "\n")

sparse_linear.replace(0, 8, 1)
sparse_linear.replace(0, 9, 1)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[1].weight_indices, "\n")

sparse_linear.replace(0, 11, 2)
print(sparse_linear.weight_indices)
print(sparse_linear.embed_linears[2].weight_indices, "\n")



tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7]]) 



ValueError: Fan in and fan out can not be computed for tensor with fewer than 2 dimensions

In [10]:
from  senmodel.metrics.nonlinearity_metrics import *


In [11]:
criterion = nn.MSELoss()

metrics = [
    GradientMeanEdgeMetric(criterion),
    PerturbationSensitivityEdgeMetric(criterion),
]

In [12]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=1000, n_features=8, n_informative=8, random_state=42)
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

dataset = list(zip(X, y))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
from senmodel.metrics.edge_finder import *

In [14]:

ef = EdgeFinder(metrics[0], test_loader)
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges(sparse_linear, 4)
print("choose:", chosen_edges)

values: tensor([212.2373,  98.3824,  80.3239, 174.1979,  73.5584,  52.2176,  35.5836,
         41.8561])
choose: tensor([[0, 0, 0, 0],
        [0, 3, 1, 2]])


In [15]:
child, parents = chosen_edges
for c, p in zip(child, parents):
    sparse_linear.replace(c, p, 3)

In [16]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 4,  5, 10, 11, 12, 13, 14, 15]])

In [17]:
ef = EdgeFinder(metrics[0], test_loader)
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges(sparse_linear, 4)
print("choose:", chosen_edges)

values: tensor([73.5880, 52.2640, 35.5937, 41.8417,  4.8890,  3.7774, 18.9784, 29.0767])
choose: tensor([[ 0,  0,  0,  0],
        [ 4,  5, 11, 10]])


In [18]:
ef = EdgeFinder(metrics[0], test_loader)
print("values:", ef.calculate_edge_metric_for_dataloader(sparse_linear))
chosen_edges = ef.choose_edges(sparse_linear, 4)
print("choose:", chosen_edges)

values: tensor([73.5880, 52.2640, 35.5937, 41.8417,  4.8890,  3.7774, 18.9784, 29.0767])
choose: tensor([[ 0,  0,  0,  0],
        [ 4,  5, 11, 10]])


In [19]:
child, parents = chosen_edges
for c, p in zip(child, parents):
    sparse_linear.replace(c, p, 3)

In [20]:
sparse_linear.weight_indices

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [12, 13, 14, 15, 16, 17, 18, 19]])